In [20]:
import pandas as pd
from sqlalchemy import create_engine


In [21]:
box_office_file = "box_office.csv"
box_office_df = pd.read_csv(box_office_file)
box_office_df.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052"
2,Fantasia,"Nov 13, 1940",Musical,G,"$83,320,000","$2,187,090,808"
3,Song of the South,"Nov 12, 1946",Adventure,G,"$65,000,000","$1,078,510,579"
4,Cinderella,"Feb 15, 1950",Drama,G,"$85,000,000","$920,608,730"


In [22]:
imdb_file = "imdb.csv"
imdb_df = pd.read_csv(imdb_file, header=None)
imdb_df.columns


Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43],
           dtype='int64')

In [23]:
reduced_box = box_office_df[['movie_title', 'release_date', 'inflation_adjusted_gross']].copy()
reduced_box = reduced_box.rename(columns={"release_date":"release_year"})
reduced_box["movie_title"] = reduced_box["movie_title"].str.upper()
box_year = reduced_box["release_year"].str.split(",")
reduced_box["release_year"] = box_year.str.get(1)
reduced_box["release_year"] = reduced_box["release_year"].str.strip()
reduced_box = reduced_box.sort_values(by="movie_title", ascending=True)
reduced_box.loc[reduced_box["release_year"] == "1992"].head()

,movie_title,release_year,inflation_adjusted_gross
133,1492: CONQUEST OF PARADISE,1992,"$14,421,454"
127,3 NINJAS,1992,"$58,965,304"
128,A GUN IN BETTY LOU'S HANDBAG,1992,"$7,295,423"
126,A STRANGER AMONG US,1992,"$24,950,752"
135,ALADDIN,1992,"$441,969,178"


In [24]:
reduced_imdb = imdb_df[[2,8,5,6,10,11]].copy()
reduced_imdb = reduced_imdb.rename(columns={2:"movie_title", 8:"release_year", 5:"imdb_rating", 6:"rating_count", 10:"wins", 11:"nominations"})
reduced_imdb = reduced_imdb.drop(reduced_imdb.index[0])
imdb_title = reduced_imdb["movie_title"].str.split("(")
reduced_imdb["movie_title"] = imdb_title.str.get(0)
reduced_imdb["movie_title"] = reduced_imdb["movie_title"].str.upper()
reduced_imdb["movie_title"] = reduced_imdb["movie_title"].str.strip()
reduced_imdb = reduced_imdb.sort_values(by="movie_title", ascending=True)
reduced_imdb.loc[reduced_imdb["movie_title"] == "TOY STORY"]

,movie_title,release_year,imdb_rating,rating_count,wins,nominations
147,TOY STORY,1995,8.3,410650,19,13


In [32]:
rds_connection_string = "postgres:2020@localhost:5432/movies"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [33]:
reduced_imdb.to_sql(name="imdb", con=engine, if_exists='replace', index=False)

In [34]:
reduced_box.to_sql(name="box_office", con=engine, if_exists='replace', index=False)